In [1]:
import importlib
import models
import helper_funtions
import preprocess
import settings

importlib.reload(settings)
# importlib.reload(models)
importlib.reload(helper_funtions)
importlib.reload(preprocess)

from models.LSTM import LSTM_HumanFi, CNN_LSTM, CNN_BiLSTM_TemporalAttention, CNN_BiLSTM_ChannelAttention, CNN_BiLSTM_DualAttention, CNN_BiLSTM_Attention
from models.RadioNet import RadioNet_NeuralWave
from models.ResNet import ECAResNet1D, ECABasicBlock1D, ResNet1D_JARILWWF, OptResNet1D_JARILWWF, OptECAResNet1D_JARILWWF, CustomResNet1D, CustomECAResNet1D
from models.TemporalConvNet import TemporalConvNet

from helper_funtions import grid_search, run_gridsearch_with_seeds
from preprocess import DataPreprocessor
from settings import folder_path_5ghz_10hz_collected, folder_path_60ghz_collected, folder_path_5ghz_200hz_collected, folder_path_60ghz_external, output_path

In [2]:
input_dim = 30
num_epochs = 500

param_grid = {
    'batch_size': [32],
    'learning_rate': [0.0007], # 0.001,
    'optimizer': ['adam'],
    'mixup_alpha': [0.0, 0.4],
    'smoothing_prob': [0.0, 0.5],
    'model': [],
}

hidden_dims = [64, 128] # 32, 256
num_layers_list = [1, 2]
bidirectional_flags = [False, True]
dropouts = [0.2, 0.5]

for hidden_dim in hidden_dims:
    for num_layers in num_layers_list:
        for bidirectional in bidirectional_flags:
            for dropout in dropouts:
                param_grid['model'].append({
                    'model_class': LSTM_HumanFi,
                    'model_args': {
                        'input_dim': input_dim,
                        'hidden_dim': hidden_dim,
                        'num_layers': num_layers,
                        'bidirectional': bidirectional,
                        'dropout': dropout
                    },
                    'num_epochs': num_epochs
                })

tcn_channel_options = [
    # [32, 64],
    [64, 128],
    [64, 128, 128],
    [64, 128, 256],
    # [128, 256, 512],
]
tcn_kernel_sizes = [2, 3] # 5
tcn_dropout_rates = [0.2, 0.5]

for channels in tcn_channel_options:
    for kernel_size in tcn_kernel_sizes:
        for dropout in tcn_dropout_rates:
            param_grid['model'].append({
                'model_class': TemporalConvNet,
                'model_args': {
                    'num_inputs': input_dim,
                    'num_channels': channels,
                    'kernel_size': kernel_size,
                    'dropout': dropout
                },
                'num_epochs': num_epochs
            })

cnn_lstm_hidden_dims = [64, 128] # 256
cnn_lstm_layers = [1, 2]

for hidden_dim in cnn_lstm_hidden_dims:
    for num_layers in cnn_lstm_layers:
        param_grid['model'].append({
            'model_class': CNN_BiLSTM_Attention,
            'model_args': {
                'input_dim': input_dim,
                'cnn_filters': 64,
                'lstm_units': hidden_dim,
                'num_layers': num_layers,
            },
            'num_epochs': num_epochs
        })

        param_grid['model'].append({
            'model_class': CNN_BiLSTM_TemporalAttention,
            'model_args': {
                'input_dim': input_dim,
                'cnn_channels': 64,
                'lstm_hidden_dim': hidden_dim,
                'lstm_layers': num_layers,
            },
            'num_epochs': num_epochs
        })

        # param_grid['model'].append({
        #     'model_class': CNN_BiLSTM_ChannelAttention,
        #     'model_args': {
        #         'input_dim': input_dim,
        #         'cnn_channels': 64,
        #         'lstm_hidden_dim': hidden_dim,
        #         'lstm_layers': num_layers,
        #     },
        #     'num_epochs': num_epochs
        # })
        #
        # param_grid['model'].append({
        #     'model_class': CNN_BiLSTM_DualAttention,
        #     'model_args': {
        #         'input_dim': input_dim,
        #         'cnn_channels': 64,
        #         'lstm_hidden_dim': hidden_dim,
        #         'lstm_layers': num_layers,
        #     },
        #     'num_epochs': num_epochs
        # })


param_grid['model'] += [
    # {
    #     'model_class': RadioNet_NeuralWave,
    #     'model_args': {'input_dim': input_dim},
    #     'num_epochs': num_epochs,
    #     'data_preprocessor': DataPreprocessor(target_dim=354)
    # },
    
    # the ones commented below are not optimal (proven by early tests)
    # {
    #     'model_class': ECAResNet1D,
    #     'model_args': {'input_channels': input_dim, 'block': ECABasicBlock1D, 'layers': (1, 1, 1, 1)},
    #     'num_epochs': num_epochs
    # },
    # {
    #     'model_class': ECAResNet1D,
    #     'model_args': {'input_channels': input_dim, 'block': ECABasicBlock1D, 'layers': (2, 2, 2, 2)},
    #     'num_epochs': num_epochs
    # },
    # {
    #     'model_class': ResNet1D_JARILWWF,
    #     'model_args': {'input_channels': input_dim, 'layers': [1,1,1,1]},
    #     'num_epochs': num_epochs,
    # },
    # {
    #     'model_class': ResNet1D_JARILWWF,
    #     'model_args': {'input_channels': input_dim, 'layers': [2,2,2,2]},
    #     'num_epochs': num_epochs,
    # },
    
    
    {
        'model_class': OptResNet1D_JARILWWF,
        'model_args': {'input_channels': input_dim, 'layers': [1,1,1,1]},
        'num_epochs': num_epochs,
    },
    # {
    #     'model_class': OptResNet1D_JARILWWF,
    #     'model_args': {'input_channels': input_dim, 'layers': [2,2,2,2]},
    #     'num_epochs': num_epochs,
    # },
    {
        'model_class': OptECAResNet1D_JARILWWF,
        'model_args': {'input_channels': input_dim, 'layers': [1,1,1,1]},
        'num_epochs': num_epochs,
    },
    # {
    #     'model_class': OptECAResNet1D_JARILWWF,
    #     'model_args': {'input_channels': input_dim, 'layers': [2,2,2,2]},
    #     'num_epochs': num_epochs,
    # },
    {
        'model_class': CustomResNet1D,
        'model_args': {'input_channels': input_dim},
        'num_epochs': num_epochs,
    },
    {
        'model_class': CustomECAResNet1D,
        'model_args': {'input_channels': input_dim},
        'num_epochs': num_epochs,
    },
] 

In [3]:
gridsearch_results_df = run_gridsearch_with_seeds(
    gridsearch_func=grid_search,
    gridsearch_args={
        'param_grid': param_grid,
        'folder_path': folder_path_60ghz_external,
        'background_subtraction': False,
        'seconds_per_sample': 5,
        'rows_per_second': 22,
    },
    n_seeds=3,
    output_dir=output_path,
    filename="gridsearch_60ghz_ext"
)


--- Running seed 42 ---
Running 160 configurations...

Processing Model: LSTM_HumanFi with params {'input_dim': 30, 'hidden_dim': 64, 'num_layers': 1, 'bidirectional': False, 'dropout': 0.2}
Started loading data...
Train size: 509, Validation size: 105, Test size: 105
Train labels distribution: Counter({4: 123, 6: 91, 0: 75, 5: 67, 1: 51, 2: 51, 3: 51})
Validation labels distribution: Counter({4: 26, 6: 19, 0: 16, 5: 14, 1: 10, 2: 10, 3: 10})
Test labels distribution: Counter({4: 26, 6: 19, 0: 16, 5: 14, 1: 10, 2: 10, 3: 10})
Done loading data (batch size: 32) in 0:00:00.046050.

Testing Configuration 0: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.0

(Device: cuda) Started training model: LSTM_HumanFi
Early stopping at epoch 210 (no improvement in 50 epochs)
Training time:  0:00:06.338176

Testing Configuration 1: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.5

(Device: cuda) Started training model: LSTM_HumanFi
Early stop

C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Early stopping at epoch 144 (no improvement in 50 epochs)
Training time:  0:02:34.531745


C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)



Testing Configuration 65: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.5

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 85 (no improvement in 50 epochs)
Training time:  0:01:33.592077


...


Testing Configuration 109: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.5

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 177 (no improvement in 50 epochs)
Training time:  0:10:24.177365


C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)



Testing Configuration 110: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.4, smoothing_prob=0.0

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 308 (no improvement in 50 epochs)
Training time:  0:18:06.480295


C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)



Testing Configuration 111: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.4, smoothing_prob=0.5

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 233 (no improvement in 50 epochs)
Training time:  0:12:29.722668

Processing Model: CNN_BiLSTM_Attention with params {'input_dim': 30, 'cnn_filters': 64, 'lstm_units': 64, 'num_layers': 1}
Started loading data...
Train size: 509, Validation size: 105, Test size: 105
Train labels distribution: Counter({4: 123, 6: 91, 0: 75, 5: 67, 1: 51, 2: 51, 3: 51})
Validation labels distribution: Counter({4: 26, 6: 19, 0: 16, 5: 14, 1: 10, 2: 10, 3: 10})
Test labels distribution: Counter({4: 26, 6: 19, 0: 16, 5: 14, 1: 10, 2: 10, 3: 10})
Done loading data (batch size: 32) in 0:00:00.016004.

Testing Configuration 112: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.0

(Device: cuda) Started training model: CNN_BiLSTM_Attention
Early stopping at epoch 108 (no improvement in 50 epochs)
Train

In [4]:
gridsearch_results_df_bgsub = run_gridsearch_with_seeds(
    gridsearch_func=grid_search,
    gridsearch_args={
        'param_grid': param_grid,
        'folder_path': folder_path_60ghz_external,
        'background_subtraction': True,
        'seconds_per_sample': 5,
        'rows_per_second': 22,
    },
    n_seeds=3,
    output_dir=output_path,
    filename="gridsearch_60ghz_ext"
)


--- Running seed 42 ---
Running 160 configurations...

Processing Model: LSTM_HumanFi with params {'input_dim': 30, 'hidden_dim': 64, 'num_layers': 1, 'bidirectional': False, 'dropout': 0.2}
Started loading data...
Train size: 509, Validation size: 105, Test size: 105
Train labels distribution: Counter({4: 123, 6: 91, 0: 75, 5: 67, 1: 51, 2: 51, 3: 51})
Validation labels distribution: Counter({4: 26, 6: 19, 0: 16, 5: 14, 1: 10, 2: 10, 3: 10})
Test labels distribution: Counter({4: 26, 6: 19, 0: 16, 5: 14, 1: 10, 2: 10, 3: 10})
Done loading data (batch size: 32) in 0:00:00.047396.

Testing Configuration 0: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.0

(Device: cuda) Started training model: LSTM_HumanFi
Early stopping at epoch 175 (no improvement in 50 epochs)
Training time:  0:00:05.714951

Testing Configuration 1: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.5

(Device: cuda) Started training model: LSTM_HumanFi
Early stop

C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Early stopping at epoch 70 (no improvement in 50 epochs)
Training time:  0:01:27.000312


C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)



Testing Configuration 65: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.5

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 81 (no improvement in 50 epochs)
Training time:  0:01:40.306314


...


Testing Configuration 109: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.5

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 70 (no improvement in 50 epochs)
Training time:  0:03:41.098852


C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)



Testing Configuration 110: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.4, smoothing_prob=0.0

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 147 (no improvement in 50 epochs)
Training time:  0:07:46.265589


C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)



Testing Configuration 111: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.4, smoothing_prob=0.5

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 149 (no improvement in 50 epochs)
Training time:  0:07:58.447423

Processing Model: CNN_BiLSTM_Attention with params {'input_dim': 30, 'cnn_filters': 64, 'lstm_units': 64, 'num_layers': 1}
Started loading data...
Train size: 509, Validation size: 105, Test size: 105
Train labels distribution: Counter({4: 123, 6: 91, 0: 75, 5: 67, 1: 51, 2: 51, 3: 51})
Validation labels distribution: Counter({4: 26, 6: 19, 0: 16, 5: 14, 1: 10, 2: 10, 3: 10})
Test labels distribution: Counter({4: 26, 6: 19, 0: 16, 5: 14, 1: 10, 2: 10, 3: 10})
Done loading data (batch size: 32) in 0:00:00.023500.

Testing Configuration 112: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.0

(Device: cuda) Started training model: CNN_BiLSTM_Attention
Early stopping at epoch 126 (no improvement in 50 epochs)
Train